In [1]:
import os

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import imageio

from numpy import float64
from matplotlib import rc
rc('text', usetex=False)

from mpl_toolkits.axisartist.parasite_axes import HostAxes, ParasiteAxes
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
from glob import glob

from ipywidgets import Layout, Button, Box
import ipywidgets as wg
from IPython.display import display

import scipy.interpolate as sip

import colour
from colour.plotting import *
from colour.colorimetry import *
from colour.models import *
from colour import SDS_ILLUMINANTS

/home/gus/anaconda3/lib/python3.8/site-packages/colour/utilities/verbose.py:237: ColourUsageWarning: "colour.models.XYZ_to_colourspace_model" object is deprecated and will be removed in a future release.
  warn(*args, **kwargs)


# Function 

In [14]:
def MFT_tab(Id,dir_path,BWS=[]):
    """
    Function that plots the micro-fading data obtained with the Tidas device

    Inputs
    ====================================================
    Id: a string
    Id number of the micro-fading analysis
    
    
    BWS: a list
    list containing filenames of the micro-fading data on BWS

    Returns
    ====================================================
    It creates a widget object with several tab showing the results
    """    

        
    d65 = colour.CCS_ILLUMINANTS["cie_10_1964"]["D65"]
    
    ###### OBTAIN THE DATA #######
    file_sp = glob(f'*{Id}*SP*')[0]
    file_dE = glob(f'*{Id}*dE*')[0]
    file_info = glob(f'*{Id}*INFO*')[0]

    df_sp = pd.read_csv(file_sp)
    df_dE = pd.read_csv(file_dE)
    df_info = pd.read_csv(file_info,index_col='parameter')

    info = file_sp.split('_')
    date = info[0]
    Id = info[1]    
    sample_type = info[2]
    pigment = df_info.loc['pigment'].values[0]
    
    if pigment == 'C.I. Acid Blue 104':
        pigment = 'BW1'
        
    if pigment == 'C.I. Acid Blue 109':
        pigment = 'BW2'
        
    if pigment == 'C.I. Acid Blue 83':
        pigment = 'BW3'
    
    wavelength = df_sp['wavelength']
    sp_i = df_sp.columns[1:].values[0]
    sp_f = df_sp.columns[-1:].values[0]

    kJ_m2 = df_dE['Radiant_exp(MJ/m2)'].values  
    lx_hr = df_dE['Exp_dose(Mlx.hr)'].values       
    dE76 = df_dE['dE76']
    dE00 = df_dE['dE00']
    dR = df_dE['dR']
    L = df_dE['L*']
    a = df_dE['a*']
    b = df_dE['b*']
    C = df_dE['C*']
    h = df_dE['h']
    t = df_dE['Time(s)']
    t_fin = int(np.round((t.values[-1]/60),0))
    lab = np.array([L, a, b]).transpose()
    srgb = colour.XYZ_to_sRGB(colour.Lab_to_XYZ(lab), d65).clip(0, 1)
    dE00_1_val_Mlx = float(df_info.loc['E_1dE00_MJ/m**2_Mlx.hr'].values[0].split('_')[1])
    dE00_1_val_MJ = float(df_info.loc['E_1dE00_MJ/m**2_Mlx.hr'].values[0].split('_')[0])    
    dE00_2_val_Mlx = float(df_info.loc['E_2dE00_MJ/m**2_Mlx.hr'].values[0].split('_')[1])
    dE00_2_val_MJ = float(df_info.loc['E_2dE00_MJ/m**2_Mlx.hr'].values[0].split('_')[0])
    photo_nb = df_info.loc['beam_photo'].values[0][-2:]
    lamp = df_info.loc['lamp'].values[0].split('_')[0]
    distance = df_info.loc['distance_ill_mm'].values[0]
    fiber = df_info.loc['fiber_ill'].values[0]
    filter = df_info.loc['filter'].values[0].split('_')[0]
    power = df_info.loc['power_mW'].values[0].split('_')[1]
    zoom =  df_info.loc['zoom'].values[0]
    device = 'Tidas'
    
    
    BS_file = glob(f'*{date}*_{photo_nb}_*BS-INFO*')[0]
    WR = BS_file.split('_')[6]
    camera = BS_file.split('_')[7]
   
    ###### CREATE THE WIDGET TABS #######
    
    out1 = wg.Output()
    out2 = wg.Output()
    out3 = wg.Output()
    out4 = wg.Output()
    out5 = wg.Output()
    out6 = wg.Output()
    out7 = wg.Output()
    out8 = wg.Output()
    out9 = wg.Output()



    tab = wg.Tab(children = [out1,out2,out3,out4,out5,out6,out7,out8,out9])
    tab.set_title(0,'Refl sp')
    tab.set_title(1,'Colour diff')
    tab.set_title(2,'Lab-LCh')
    tab.set_title(3,'CIELAB space')
    tab.set_title(4,'Colour patches')
    tab.set_title(5, 'Colour change slider')
    tab.set_title(6,'Data')
    tab.set_title(7,'Fading beam')
    tab.set_title(8,'Parameters')

    display(tab)
    
    ###### SET PARAMATERS #######

    fts1 = 21
    fts2 = 15
    fts_leg = 21
    s = 30
    
    
    ###### CREATE SAVE BUTTON #######
    
    
    b2 = wg.Button(description='Save plot')
    vbox2 = wg.VBox([b2])
    vbox2bis = wg.Output()
    
    b3 = wg.Button(description='Save plot')
    vbox3 = wg.VBox([b3])
    vbox3bis = wg.Output()
    
    b4 = wg.Button(description='Save plot')
    vbox4 = wg.VBox([b4])
    vbox4bis = wg.Output()
    
    b5 = wg.Button(description='Save plot')
    vbox5 = wg.VBox([b5])
    vbox5bis = wg.Output()
    

    ###### FILL EACH TAB #######
    
    with out1:
        sns.set()

        fig1,ax1 = plt.subplots(1,1, figsize = (15,9))

        ax1.plot(wavelength,df_sp[sp_i], label = 'initial')
        ax1.plot(wavelength,df_sp[sp_f], '--', label = 'final')

        ax1.xaxis.set_tick_params(labelsize=21)
        ax1.yaxis.set_tick_params(labelsize=21)

        ax1.set_xlim(wavelength.values[0],wavelength.values[-1])
        ax1.set_ylim(bottom = 0)

        ax1.set_xlabel('Wavelength $\lambda$ (nm)', fontsize = fts1)
        ax1.set_ylabel('Reflectance factor  (%)', fontsize = fts1) 

        name = fr'Micro-fading analysis - Reflectance spectra, {date}, {Id}, $\bf{pigment}$'
        ax1.set_title(name,fontsize = 23)
        ax1.legend(loc='best', fontsize= fts_leg)
        plt.tight_layout()
        plt.show()
        
      
        


    with out2:
        fig2,ax2 = plt.subplots(1,1, figsize = (15,9))

        ax3 = ax2.twiny()

        ax2.plot(kJ_m2,dE76, lw = 0.5, color = 'blue', label = r'$\Delta E^*_{ab}$')
        ax2.plot(kJ_m2,dE00, lw = 0.5, color = 'red', label = r'$\Delta E^*_{00}$')
        #ax2.plot(kJ_m2,dR, lw = 0.5, ls = '--', color = 'green', label = r'$\Delta R$')

        ax3.plot((lx_hr),dE76, lw = 0.8, color = 'blue',label = r'$\Delta E^*_{ab}$')
        ax3.plot((lx_hr),dE00, lw = 0.8, color = 'red',label = r'$\Delta E^*_{00}$')
        ax3.plot((lx_hr),dR, ls = '--', lw = 0.8, color = 'green', label = r'$\Delta R$')

        ax2.xaxis.set_tick_params(labelsize=fts1)
        ax3.xaxis.set_tick_params(labelsize=fts1)
        ax2.yaxis.set_tick_params(labelsize=fts1)

        ax2.set_xlim(left = 0)
        ax3.set_xlim(left = 0)
        ax3.set_ylim(bottom = 0)
        ax3.grid(False)

        ax2.set_xlabel('Radiant Exposure (MJ/m²)', fontsize = fts1)
        ax3.set_xlabel('Exposure dose (Mlx.hr)', fontsize = fts1)
        ax2.set_ylabel(r'$\Delta$', fontsize = fts1) 
        
        name = fr'Micro-fading analysis - Colour difference curves, {date}, {Id}, $\bf{pigment}$'
        ax2.set_title(name,fontsize = 23, y =1.12)

        plt.legend(loc='best', fontsize = fts_leg)
        plt.tight_layout(rect = (0,0,1,0.98))
        plt.show()
        


    with out3:
        fig3, ax4 = plt.subplots(3,1, figsize = (10,10))

        ax4[0].plot(t/60,L, color = 'blue')

        ax4[0].set_xlim(left = 0)
        ax4[0].set_ylabel(r'Brigthness (CIE $L^*$)', color = 'blue', fontsize = fts2)
        ax4[0].xaxis.set_tick_params(labelsize=fts2)
        ax4[0].yaxis.set_tick_params(labelsize=fts2)   
        for tl in ax4[0].get_yticklabels():
            tl.set_color('blue')   

        ax5 = ax4[1].twinx()    
        ax4[1].plot(t/60,a, color  ='red')
        ax5.plot(t/60,b, color = 'black')

        for tl in ax4[1].get_yticklabels():
            tl.set_color('red')

        for tl in ax5.get_yticklabels():
            tl.set_color('black')

        ax5.grid(False)
        ax4[1].set_xlim(left = 0)
        ax4[1].set_ylabel(r'CIE $a^*$', fontsize = fts2, color = 'red')
        ax5.set_ylabel(r'CIE $b^*$', fontsize = fts2)
        ax4[1].xaxis.set_tick_params(labelsize=fts2)
        ax4[1].yaxis.set_tick_params(labelsize=fts2) 
        ax5.yaxis.set_tick_params(labelsize=fts2) 

        ax6 = ax4[2].twinx()
        ax4[2].plot(t/60,C, color = 'red')
        ax6.plot(t/60,h, color = 'black')

        ax4[2].set_xlim(left = 0)

        for tl in ax4[2].get_yticklabels():
            tl.set_color('red')

        for tl in ax6.get_yticklabels():
            tl.set_color('black')

        ax6.grid(False)
        ax4[2].set_xlabel(r'Exposure time (min)', fontsize = fts2)
        ax4[2].set_ylabel(r'Saturation (CIE $C^*$)', fontsize = fts2, color = 'red')
        ax6.set_ylabel(r'Hue (CIE $h$)', fontsize = fts2)
        ax4[2].xaxis.set_tick_params(labelsize=fts2)
        ax4[2].yaxis.set_tick_params(labelsize=fts2) 
        ax6.yaxis.set_tick_params(labelsize=fts2)
        
        name = 'Micro-fading analysis - Colorimetric values \n' + fr'{date}, {Id}, $\bf{pigment}$'
        
        plt.suptitle(name,fontsize = 23)
        plt.tight_layout(rect = (0,0,1,0.99))
        plt.show()
        
        
    with out4:
        fig4, ax = plt.subplots(2,2, figsize=(10, 10), gridspec_kw=dict(width_ratios=[1, 2], height_ratios=[2, 1]))
  
        Lb = ax[0, 0]
        ab = ax[0, 1]
        AB = ax[1,0]
        aL = ax[1, 1]
        
        
        
        Lb.scatter(L[0], b[0], marker = 'x', s = 40, color=srgb[0])
        ab.scatter(a[0], b[0], marker = 'x', s = 40, color=srgb[0], label = 'start point')
        aL.scatter(a[0], L[0], marker = 'x', s = 40, color=srgb[0])        
        
        Lb.scatter(L[1:], b[1:],s = s, color=srgb[1:])
        ab.scatter(a[1:], b[1:],s = s, color=srgb[1:])
        AB.scatter(a, b,s = s, color=srgb)
        aL.scatter(a[1:], L[1:],s = s, color=srgb[1:])
        
        Lb.axhline(0, color="black", lw=0.5)
        ab.axhline(0, color="black", lw=0.5)
        AB.axhline(0, color="black", lw=0.5)
        aL.axhline(50, color="black", lw=0.5)

        Lb.axvline(50, color="black", lw=0.5)
        ab.axvline(0, color="black", lw=0.5)
        AB.axvline(0, color="black", lw=0.5)
        aL.axvline(0, color="black", lw=0.5)
        
        
        Lb.set_xlim(L.min()-0.5, L.max()+0.5)
        Lb.set_ylim(b.min()-0.5, b.max()+0.5)
        ab.set_xlim(a.min()-0.5, a.max()+0.5)
        ab.set_ylim(b.min()-0.5, b.max()+0.5) 
        AB.set_xlim(-100, 100)
        AB.set_ylim(-100, 100)  
        aL.set_xlim(a.min()-0.5, a.max()+0.5)
        aL.set_ylim(L.min()-0.5, L.max()+0.5)       
                     
        Lb.set_xlabel("CIE $L^*$", fontsize=fts2)
        Lb.set_ylabel("CIE $b^*$", fontsize=fts2)
        AB.set_xlabel("CIE $a^*$", fontsize=fts2)
        AB.set_ylabel("CIE $b^*$", fontsize=fts2)
        aL.set_xlabel("CIE $a^*$", fontsize=fts2)
        aL.set_ylabel("CIE $L^*$", fontsize=fts2) 
        
        
        Lb.xaxis.set_tick_params(labelsize=fts2)
        Lb.yaxis.set_tick_params(labelsize=fts2)
        ab.xaxis.set_tick_params(labelsize=fts2)
        ab.yaxis.set_tick_params(labelsize=fts2)   
        AB.xaxis.set_tick_params(labelsize=fts2)
        AB.yaxis.set_tick_params(labelsize=fts2)
        aL.xaxis.set_tick_params(labelsize=fts2)
        aL.yaxis.set_tick_params(labelsize=fts2)
                      
     
        red = matplotlib.patches.Rectangle((90,-7), 14, 14, color='red', alpha = 1)
        yellow = matplotlib.patches.Rectangle((-7,90), 14, 14, color='yellow', alpha = 1)
        green = matplotlib.patches.Rectangle((-100,-7), 14, 14, color='green', alpha = 1)
        blue = matplotlib.patches.Rectangle((-7,-100), 14, 14, color='blue', alpha = 1)
        black = matplotlib.patches.Rectangle((a.min()-10,b.min()-10), (a.max()-a.min())+20, (b.max()-b.min())+20,fill=None, ls = '--', color='black', alpha = 1)
        AB.add_patch(red)
        AB.add_patch(yellow)
        AB.add_patch(green)
        AB.add_patch(blue)
        AB.add_patch(black)
        
        ab.legend(loc = 'best')
        
        name = 'Micro-fading analysis - CIELAB space \n' + fr'{date}, {Id}, $\bf{pigment}$'
               
        plt.suptitle(name,fontsize = 23)
        plt.tight_layout(rect = (0,0,1,0.99))
        plt.show()



    with out5:
        fig5, axes = plt.subplots(2,1,figsize = (15,7))
        
        fig5.patch.set_facecolor((0.75, 0.75, 0.75))
        
        kJ_m2_last = kJ_m2[-1]  # in MJ/m²
        Mlx_hr_last = lx_hr[-1] # in Mlx.hr

        x = np.linspace(0,len(srgb),num = 5, endpoint = True,dtype = 'int')
        #t_int = np.linspace(0,t_fin,num = 5, endpoint = True,dtype = 'int')
        MJ_m2_values = np.linspace(0,kJ_m2_last, num = 5, endpoint = True, dtype = 'int')
        Mlx_hr_values = np.round(np.linspace(0,Mlx_hr_last, num = 5, endpoint = True, dtype = 'float'),2)

        cp_0 = matplotlib.patches.Rectangle((0,0), 0.2, 1, color=srgb[0])
        cp_1 = matplotlib.patches.Rectangle((0.2,0), 0.2, 1, color=srgb[x[1]])
        cp_2 = matplotlib.patches.Rectangle((0.4,0), 0.2, 1, color=srgb[x[2]])
        cp_3 = matplotlib.patches.Rectangle((0.6,0), 0.2, 1, color=srgb[x[3]])
        cp_4 = matplotlib.patches.Rectangle((0.8,0), 0.2, 1, color=srgb[-1])
        axes[0].add_patch(cp_0)
        axes[0].add_patch(cp_1)    
        axes[0].add_patch(cp_2) 
        axes[0].add_patch(cp_3) 
        axes[0].add_patch(cp_4) 
        axes[0].grid(False)
        axes[0].axis('off')
        
        if L[0] > 50:
            axes[0].annotate('0 MJ/m²',(0.06, 0.05),fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[1]} MJ/m²',(0.25, 0.05),fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[2]} MJ/m²',(0.45, 0.05),fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[3]} MJ/m²',(0.65, 0.05),fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[4]} MJ/m²',(0.85, 0.05),fontsize = fts1)            
            axes[0].annotate('0 Mlx.hr',(0.06, 0.9),weight='bold',fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[1]} Mlx.hr',(0.25, 0.9),fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[2]} Mlx.hr',(0.45, 0.9),fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[3]} Mlx.hr',(0.65, 0.9),fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[4]} Mlx.hr',(0.85, 0.9),fontsize = fts1)  
            
        else:
            axes[0].annotate('0 MJ/m²',(0.06, 0.05),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[1]} MJ/m²',(0.25, 0.05),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[2]} MJ/m²',(0.45, 0.05),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[3]} MJ/m²',(0.65, 0.05),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{MJ_m2_values[4]} MJ/m²',(0.85, 0.05),color = 'white',fontsize = fts1)  
            axes[0].annotate('0 Mlx.hr',(0.06, 0.9),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[1]} Mlx.hr',(0.25, 0.9),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[2]} Mlx.hr',(0.45, 0.9),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[3]} Mlx.hr',(0.65, 0.9),color = 'white',fontsize = fts1)
            axes[0].annotate(f'{Mlx_hr_values[4]} Mlx.hr',(0.85, 0.9),color = 'white',fontsize = fts1) 

        cp_i = matplotlib.patches.Rectangle((0,0), 0.5, 1, color=srgb[0])
        cp_f = matplotlib.patches.Rectangle((0.5,0), 0.5, 1, color=srgb[-1])
        axes[1].add_patch(cp_i)
        axes[1].add_patch(cp_f)    
        axes[1].grid(False)
        axes[1].axis('off')

        if L[0] > 50:
            axes[1].annotate('Initial - 0 min',(0.02, 0.05),fontsize = fts1)
            axes[1].annotate(f'Final - {t_fin} min',(0.8, 0.05),fontsize = fts1)
            
        else:
            axes[1].annotate('Initial - 0 min',(0.02, 0.05),color = 'white',fontsize = fts1)
            axes[1].annotate(f'Final - {t_fin} min',(0.8, 0.05),color = 'white', fontsize = fts1) 
            
        name = 'Micro-fading analysis - Colour patches \n' + fr'{date}, {Id}, $\bf{pigment}$'
        
        plt.suptitle(name,fontsize = 23)
        #plt.tight_layout(rect = (0,0,1,0.95))
        plt.show()
        

    with out6:
        style = {'description_width': 'initial'}
        L_i = L[0]
        a_i = a[0]
        b_i = b[0]
        Lab_i = (L_i,a_i,b_i) 
        sRGB_i = colour.XYZ_to_sRGB(colour.Lab_to_XYZ(Lab_i), d65).clip(0, 1)
        
        dose = lx_hr
        max_dose = dose.max()

        def f(ill_slider,t_slider,hpd):

            range_dose = np.arange(0,dose[-1],1)
            range_dE = np.arange(0,dE00.values[-1],0.01)

            f_dose = sip.interp1d(dE00,dose)
            f_dE = sip.interp1d(dose,dE00)
            f_L = sip.interp1d(dose,L)
            f_a = sip.interp1d(dose,a)
            f_b = sip.interp1d(dose,b)

            f_L(range_dose)
            f_a(range_dose)
            f_b(range_dose)    
            f_dE(range_dose)
            f_dose(range_dE)


            days = int(t_slider/hpd)
            d = np.round((ill_slider/1000000)*t_slider,2)
            

            if d <= max_dose:

                dE = np.round(f_dE(d),2)
                L_val = np.round(f_L(d),2)
                a_val = np.round(f_a(d),2)
                b_val = np.round(f_b(d),2)

                Lab_val = (L_val,a_val,b_val)    
                sRGB = colour.XYZ_to_sRGB(colour.Lab_to_XYZ(Lab_val), d65).clip(0, 1)

                fig7, ax8 = plt.subplots(figsize=(12, 7),facecolor = '#BFBFBF')


                cp_i = matplotlib.patches.Rectangle((0,0), 0.5, 1, color=(sRGB_i[0],sRGB_i[1],sRGB_i[2]))
                cp_f = matplotlib.patches.Rectangle((0.5,0), 0.5, 1, color=(sRGB[0],sRGB[1],sRGB[2]))
                ax8.add_patch(cp_i)
                ax8.add_patch(cp_f)
                ax8.set_xlim(-0.05, 1.05)
                ax8.set_ylim(-0.1, 1.1)
                ax8.axis('off')        

                ax8.annotate('Initial colour',(0.02, 0.05),fontsize = 20)
                ax8.annotate('Estimated colour',(0.7, 0.05),fontsize = 20)
                ax8.set_title(r'$\Delta E^*_{00}$' + f' = {dE}, Days of exposure = {days}, Exposure dose = {d} Mlx.hr', fontsize = 18, y = 0.99)

                plt.show()

            else:
                print('The given exposure exceeds the maximum dose for which data is available.')

            b6 = wg.Button(description='Save image')
            vbox6 = wg.VBox([b6])
            vbox6bis = wg.Output()

            def button(_):
                with vbox6bis:                    
                    fig7.savefig(dir_path+f'{Id}_{pigment}_cp.png',facecolor = (0.75, 0.75, 0.75))
                    print('figure saved !')

            b6.on_click(button)

            display(wg.HBox([vbox6, vbox6bis]))

        ill_slider = wg.IntSlider(description='Illuminance (lux)', value = 100,min = 0, max = 500, step = 1,style = style,layout=Layout(width='47%', height='30px'))
        t_slider = wg.IntSlider(description='Duration (hrs)', min = 0, max = 10000, step = 1, style = style,layout=Layout(width='47%', height='30px'))
        hpd = wg.IntText(description = 'Hours per day', value = 10, max = 24, step = 1, style = style, layout=Layout(width='15%', height='30px'))

        interactive_plot = wg.interactive(f,ill_slider = ill_slider, t_slider = t_slider, hpd = hpd)
        output = interactive_plot.children[-1]
        output.layout.height = '550px'
        display(interactive_plot)
        
        
    with out7:
        display(df_dE)

    with out8:
             
        
        BS_plot_file = glob(f'*{date}*{photo_nb}*BS-PLOT*')[0]
        BS_irr_file = glob(f'*{date}*{photo_nb}*BS-IRR*')[0]
        
        #BS_plot_file = path_fig_BS+f'{date}_{photo_nb}_{lamp}_{filter}_{distance}mm_{fiber_short}um_{WR}_{camera}_BS-PLOT.png'
        #BS_irr_file = path_fig_BS+f'{date}_{photo_nb}_{lamp}_{filter}_{distance}mm_{fiber_short}um_{WR}_{camera}_BS-IRR.png'
        fig, ax6 = plt.subplots(1,2,figsize = (45,30))
        
        im1 = imageio.imread(BS_irr_file)
        ax6[0].imshow(im1)
        ax6[0].grid(False)
        ax6[0].axis('off')
        
        im2 = imageio.imread(BS_plot_file)
        ax6[1].imshow(im2)
        ax6[1].grid(False)
        ax6[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
    with out9:
        display(df_info)




## Data

In [15]:
dir_path = '/home/gus/Documents/informatique/Github/'
MFT_tab('MF.0055.08', dir_path)